**ENSF 612 - Fall 2022  
Term Project  
Sentiment Analysis of Haribo Gummy Bear Review Comments**  
Group 15  
Chansuck (Shawn) Lee  
Nicolas Hirschfeld  
Tania So

**Load and view data**

In [0]:
file_location = "dbfs:/FileStore/shared_uploads/hirschfeldnic@gmail.com/datasetlabelled.csv"
file_type = "csv"

df_data = spark.read.format(file_type).option("multiline", "true").option("quote", '"').option("header", "true").option("escape", "\\").option("escape", '"').load(file_location)

In [0]:
display(df_data)

isVerified,position,productAsin,ratingScore,_c4,reviewDescription,reviewImages/0,reviewImages/1,reviewImages/2,reviewImages/3,reviewImages/4,reviewImages/5,reviewImages/6,reviewImages/7,reviewReaction,reviewTitle,reviewUrl,reviewedIn,variant,_c19
true,1075,B000EVOSE4,2,2,"I recently ordered a 5 pound bag of Goldbears, which came in the original gold bag and says “Made in Turkey.” Those were great and I truly enjoyed them...5 stars.This time, I ordered the same product, but received a clear bag of “Goldbears” that says ”Made in Brazil.” I am very disappointed, and almost feel as if I’ve been scammed. These bears are much softer in texture than the gold bag, taste is disappointing combared to the gold bag, and are overall inferior to the “Made in Turkey” gold bag product.I won’t re-order these anytime soon, as it is a gamble as to which product I receive.",https://images-na.ssl-images-amazon.com/images/I/61b9Er3ECAL._SY88.jpg,null,null,null,null,null,null,null,One person found this helpful,No guarantee which version of product or source. Feel ripped off...,https://www.amazon.com/gp/customer-reviews/R1A5J7823SRRR/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B000EVOSE4,"Reviewed in the United States on February 11, 2020",Flavor Name: Goldbears,0.790949954
true,76,B000EVOSE4,5,3,"Used this big bag to fill a Mason jar for a baby shower ""count the cubies"" super cute and colorful.",null,null,null,null,null,null,null,null,null,Love gummy bears,https://www.amazon.com/gp/customer-reviews/R1Y0WK8NO2H308?ASIN=B000EVOSE4,"Reviewed in the United States on July 29, 2022",Flavor Name: Goldbears 3 lbs.,0.26492906
true,2044,B000EVOSE4,2,1,"If you get a Haribo from grocery store it'll be made in Germany and delicious. These are made in Brazil and don't taste the same at all. Packaging is just a transparent bag, no colorful branding.",null,null,null,null,null,null,null,null,One person found this helpful,It's not same with the grocery store item,https://www.amazon.com/gp/customer-reviews/R351D0LLBVPTCB/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B000EVOSE4,"Reviewed in the United States on October 28, 2020",Flavor Name: Goldbears,0.283637858
true,1987,B000EVOSE4,4,3,I'm not the only one who likes these golden bears,https://images-na.ssl-images-amazon.com/images/I/81FR4PXS1CL._SY88.jpg,null,null,null,null,null,null,null,One person found this helpful,Four Stars,https://www.amazon.com/gp/customer-reviews/ROIZ5Q09WWZLL/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B000EVOSE4,"Reviewed in the United States on February 10, 2018",Flavor Name: Goldbears,0.585693443
true,3829,B000EVOSE4,4,3,Very good. Only 4 stars due to the lack of a resealable bag.,null,null,null,null,null,null,null,null,null,Very good candy,https://www.amazon.com/gp/customer-reviews/R24OAML1NZ9VHZ/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B000EVOSE4,"Reviewed in the United States on July 2, 2014",Flavor Name: Goldbears,0.444979213
true,4747,B000EVOSE4,1,1,"I ordered 2 bags of the haribo gummy bears and was not very happy. When i first opened the package, I noticed something was not quite right. I thought it didn't quite look like 10 pounds. 8.26 was the real weight. I was very upset because I was very excited. I started to become very enraged and wanted my money back. They are also very chewy and are very unflavorful. Please send me a new order of 2 gummybears and I hope to have buisiness with you again.Regards,Charlie",null,null,null,null,null,null,null,null,4 people found this helpful,Unacceptable,https://www.amazon.com/gp/customer-reviews/RGB9MTXY28YLV?ASIN=B000EVOSE4,"Reviewed in the United States on January 17, 2013",Flavor Name: Goldbears,0.469542388
true,1699,B000EVOSE4,4,3,They taste normal and they were packaged good,null,null,null,null,null,null,null,null,null,There's a lot of the gummy bears!!!!,https://www.amazon.com/gp/customer-reviews/RJJE9IHBUM31H/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B000EVOSE4,"Reviewed in the United States on January 3, 2019",Flavor Name: Goldbears,0.172785701
true,1087,B000EVOSE4

In [0]:
df_data.printSchema()

root
 |-- isVerified: string (nullable = true)
 |-- position: string (nullable = true)
 |-- productAsin: string (nullable = true)
 |-- ratingScore: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- reviewDescription: string (nullable = true)
 |-- reviewImages/0: string (nullable = true)
 |-- reviewImages/1: string (nullable = true)
 |-- reviewImages/2: string (nullable = true)
 |-- reviewImages/3: string (nullable = true)
 |-- reviewImages/4: string (nullable = true)
 |-- reviewImages/5: string (nullable = true)
 |-- reviewImages/6: string (nullable = true)
 |-- reviewImages/7: string (nullable = true)
 |-- reviewReaction: string (nullable = true)
 |-- reviewTitle: string (nullable = true)
 |-- reviewUrl: string (nullable = true)
 |-- reviewedIn: string (nullable = true)
 |-- variant: string (nullable = true)
 |-- _c19: string (nullable = true)



**Preprocess data**

In [0]:
# imports
import pyspark.sql.functions as f
from pyspark.sql.functions import lit
from pyspark.sql.types import *
from pyspark.sql.functions import concat_ws
from pyspark.ml.feature import StopWordsRemover
!pip install pyspellchecker
from spellchecker import SpellChecker
spell = SpellChecker()
!pip install nltk
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-85faae0f-505b-48e6-ad6e-be0a2567e7b7/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-85faae0f-505b-48e6-ad6e-be0a2567e7b7/bin/python -m pip install --upgrade pip' command.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [0]:
# NOISE REDUCTION - USED IN preprocessFile(df)
def noiseReduction(words_list):
    noise_reduction = [word.lower() for word in words_list if word.isalpha() and len(word) > 2]
    
    return " ".join(noise_reduction)

udfNoiseReduction = udf(lambda z: noiseReduction(z))

# SPELL CHECK - USED IN preprocessFile(df)
def spellCheck(words):
    misspelled = spell.unknown(words.split())
    corrected_list = []
    for word in words.split():
        if word not in misspelled:
            corrected_list.append(word)
        if word in misspelled:
            corrected_list.append(spell.correction(word))
    return corrected_list
    
udfSpellCheck = udf(lambda z: spellCheck(z))

# LEMMATIZATION - USED IN preprocessFile(df)
def lemmatization(text):
    if not text:
        return ""
    
    lemmatized_words = [lemmatizer.lemmatize(word) for word in text if word is not None]
    
    return " ".join(lemmatized_words)

udfLemmatization = udf(lambda z: lemmatization(z))


# PREPROCESS METHOD
def preprocessFile(df):
    # combine Title and Body
    df_combined = df.withColumn("Combined", concat_ws(' ', df.reviewDescription, df.reviewTitle))
    
    # removing stop words
    df_combined = df_combined.withColumn("tokens", f.split("Combined", "\W+"))
    remover = StopWordsRemover(inputCol="tokens", outputCol="No Stops")
    no_stops = remover.transform(df_combined)
    
    # noise reduction (removing all non-alphabetic characters including punctuation and words less than 3 chars)
    no_noise = no_stops.withColumn("No Noise", udfNoiseReduction("No Stops"))
    
    # spell checking
    spell_check = no_noise.withColumn("Spell Check", udfSpellCheck("No Noise"))
    
    # lemmatization
    result = spell_check.withColumn("Preprocessed", udfLemmatization("Spell Check"))
    
    return result

In [0]:
df_preprocess = preprocessFile(df_data)
display(df_preprocess)

isVerified,position,productAsin,ratingScore,_c4,reviewDescription,reviewImages/0,reviewImages/1,reviewImages/2,reviewImages/3,reviewImages/4,reviewImages/5,reviewImages/6,reviewImages/7,reviewReaction,reviewTitle,reviewUrl,reviewedIn,variant,_c19,Combined,tokens,No Stops,No Noise,Spell Check,Preprocessed
true,1075,B000EVOSE4,2,2,"I recently ordered a 5 pound bag of Goldbears, which came in the original gold bag and says “Made in Turkey.” Those were great and I truly enjoyed them...5 stars.This time, I ordered the same product, but received a clear bag of “Goldbears” that says ”Made in Brazil.” I am very disappointed, and almost feel as if I’ve been scammed. These bears are much softer in texture than the gold bag, taste is disappointing combared to the gold bag, and are overall inferior to the “Made in Turkey” gold bag product.I won’t re-order these anytime soon, as it is a gamble as to which product I receive.",https://images-na.ssl-images-amazon.com/images/I/61b9Er3ECAL._SY88.jpg,null,null,null,null,null,null,null,One person found this helpful,No guarantee which version of product or source. Feel ripped off...,https://www.amazon.com/gp/customer-reviews/R1A5J7823SRRR/ref=cm_cr_arp_d_rvw_ttl?ie=UTF8&ASIN=B000EVOSE4,"Reviewed in the United States on February 11, 2020",Flavor Name: Goldbears,0.790949954,"I recently ordered a 5 pound bag of Goldbears, which came in the original gold bag and says “Made in Turkey.” Those were great and I truly enjoyed them...5 stars.This time, I ordered the same product, but received a clear bag of “Goldbears” that says ”Made in Brazil.” I am very disappointed, and almost feel as if I’ve been scammed. These bears are much softer in texture than the gold bag, taste is disappointing combared to the gold bag, and are overall inferior to the “Made in Turkey” gold bag product.I won’t re-order these anytime soon, as it is a gamble as to which product I receive. No guarantee which version of product or source. Feel ripped off...","List(I, recently, ordered, a, 5, pound, bag, of, Goldbears, which, came, in, the, original, gold, bag, and, says, Made, in, Turkey, Those, were, great, and, I, truly, enjoyed, them, 5, stars, This, time, I, ordered, the, same, product, but, received, a, clear, bag, of, Goldbears, that, says, Made, in, Brazil, I, am, very, disappointed, and, almost, feel, as, if, I, ve, been, scammed, These, bears, are, much, softer, in, texture, than, the, gold, bag, taste, is, disappointing, combared, to, the, gold, bag, and, are, overall, inferior, to, the, Made, in, Turkey, gold, bag, product, I, won, t, re, order, these, anytime, soon, as, it, is, a, gamble, as, to, which, product, I, receive, No, guarantee, which, version, of, product, or, source, Feel, ripped, off, )","List(recently, ordered, 5, pound, bag, Goldbears, came, original, gold, bag, says, Made, Turkey, great, truly, enjoyed, 5, stars, time, ordered, product, received, clear, bag, Goldbears, says, Made, Brazil, disappointed, almost, feel, ve, scammed, bears, much, softer, texture, gold, bag, taste, disappointing, combared, gold, bag, overall, inferior, Made, Turkey, gold, bag, product, won, re, order, anytime, soon, gamble, product, receive, guarantee, version, product, source, Feel, ripped, )",recently ordered pound bag goldbears came original gold bag says made turkey great truly enjoyed stars time ordered product received clear bag goldbears says made brazil disappointed almost feel scammed bears much softer texture gold bag taste disappointing combared gold bag overall inferior made turkey gold bag product won order anytime soon gamble product receive guarantee version product source feel ripped,"[recently, ordered, pound, bag, goldberg, came, original, gold, bag, says, made, turkey, great, truly, enjoyed, stars, time, ordered, product, received, clear, bag, goldberg, says, made, brazil, disappointed, almost, feel, scammed, bears, much, softer, texture, gold, bag, taste, disappointing, compared, gold, bag, overall, inferior,

**Pipeline**

In [0]:
# creating dataframe with the relevant columns Preprocessed and Label
# Specific for CSV files, Mac have preset _c4 column but PC's do not
df_data = df_preprocess.select(f.col('Preprocessed'), f.col('_c4').cast("Int").alias('Label'))
display(df_data)

Preprocessed,Label
recently ordered pound bag goldberg came original gold bag say made turkey great truly enjoyed star time ordered product received clear bag goldberg say made brazil disappointed almost feel scammed bear much softer texture gold bag taste disappointing compared gold bag overall inferior made turkey gold bag product won order anytime soon gamble product receive guarantee version product source feel ripped,2
used big bag fill mason jar baby shower count cube super cute colorful love gummy bear,3
get harbor grocery store made germany delicious made brazil taste packaging transparent bag colorful branding grocery store item,1
one like golden bear four star,3
good star due lack repeatable bag good candy,3
ordered bag harbor gummy bear happy first opened package noticed something quite right thought didn quite look like pound real weight upset excited started become enraged wanted money back also chewy flavorful please send new order hope business regard charlie unacceptable,1
taste normal packaged good lot gummy bear,3
one received little harder one find store careful order hot climate melt little perfect bag person find little bag store lacking watch heat,3
teacher purchased several month ago give student turned exit card gummy bear soft fresh package received yesterday two month past best date gummy bear stale tasting hard amazon gave refund immediately appreciate past best date check package serving,1
four star,2


In [0]:
# imports
from pyspark.ml.feature import Tokenizer, CountVectorizer, HashingTF, IDF
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, NaiveBayes, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

**Pipeline Implementation 1: Bag of Words + Logistic regression**

In [0]:
Pipeline_1 = "Bag of Words + Logistic Regression"
# Feature extraction (Bag of Words)
tokenizer_1 = Tokenizer(inputCol='Preprocessed',outputCol='Tokens')
vectorizer_1 = CountVectorizer(inputCol='Tokens',outputCol='Features')

# Split data set into train and test sets
train_data, test_data = df_data.randomSplit((0.8,0.2), seed=42)

# ML model 
lr_1 = LogisticRegression(featuresCol='Features',labelCol='Label')

# Pipeline
pipeline_1 = Pipeline(stages=[tokenizer_1, vectorizer_1, lr_1])

# Tuning Parameters
paramGrid_1 = (ParamGridBuilder()
        .addGrid(lr_1.regParam, [0.0, 0.3, 0.5]) # regularization parameter (default is 0)
        .addGrid(lr_1.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (default is 0)
        .build())

# Multiclass Classification Evaluator    
evaluator_1 = MulticlassClassificationEvaluator(labelCol='Label',
                                              predictionCol='prediction',
                                              metricName='accuracy')

# Cross Validator - creating with 2 folds
crossVal_1 = CrossValidator(estimator = pipeline_1, 
                          estimatorParamMaps = paramGrid_1, 
                          evaluator = evaluator_1, 
                          numFolds = 2)
# Implementation
model_1 = crossVal_1.fit(train_data)

# Predictions
predictions_1 = model_1.transform(test_data)

# Accuracy Score
accuracy_model_1 = evaluator_1.evaluate(predictions_1)

print(accuracy_model_1)

0.5679012345679012


In [0]:
# pulling the hyper parameter selected
bestModel_1 = model_1.bestModel
reg_param_value_1 = bestModel_1.stages[2].getRegParam()
elastic_net_value_1 = bestModel_1.stages[2].getElasticNetParam()
hyper_parameter_1 = [reg_param_value_1, elastic_net_value_1]
result_1 = ("regParam: "+ str(hyper_parameter_1[0])+", elasticNetParam: "+ str(hyper_parameter_1[1]))
print(result_1)

regParam: 0.3, elasticNetParam: 0.1


**Pipeline Implementation 2: Bag of Words + Naive Bayes**

In [0]:
Pipeline_2 = "Bag of Words + Naive Bayes"
# Feature extraction (Bag of Words)
tokenizer_2 = Tokenizer(inputCol='Preprocessed',outputCol='Tokens')
vectorizer_2 = CountVectorizer(inputCol='Tokens',outputCol='Features')

# Split data set into train and test sets
train_data, test_data = df_data.randomSplit((0.8,0.2), seed=42)

# ML model for Naive Bayes
nb_2 = NaiveBayes(featuresCol='Features',labelCol='Label')

# Pipeline for bag of words and naive bayes
pipeline_2 = Pipeline(stages=[tokenizer_2, vectorizer_2, nb_2])

# Tuning Parameters - adjust smoothing
paramGrid_2 = (ParamGridBuilder()
        .addGrid(nb_2.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]) # Default smoothing = 1.0
        .build())

# Multiclass Classification Evaluator    
evaluator_2 = MulticlassClassificationEvaluator(labelCol='Label',
                                              predictionCol='prediction',
                                              metricName='accuracy')

# Cross Validator - creating with 2 folds
crossVal_2 = CrossValidator(estimator = pipeline_2, 
                          estimatorParamMaps = paramGrid_2, 
                          evaluator = evaluator_2, 
                          numFolds = 2)
# Implementation
model_2 = crossVal_2.fit(train_data)

# Predictions
predictions_2 = model_2.transform(test_data)

# Accuracy Score
accuracy_model_2 = evaluator_2.evaluate(predictions_2)

print(accuracy_model_2)

0.5864197530864198


In [0]:
# pulling the hyper parameter selected
bestModel_2 = model_2.bestModel
smoothing_value_2 = bestModel_2.stages[2].getSmoothing()
hyper_parameter_2 = [smoothing_value_2]
result_2 = ("smoothing: "+ str(hyper_parameter_2[0]))
print(result_2)

smoothing: 1.0


**Pipeline Implementation 3: Bag of Words + RandomForest**

In [0]:
Pipeline_3 = "Bag of Words + Random Forest"
# Feature extraction (Bag of Words)
tokenizer_3 = Tokenizer(inputCol='Preprocessed',outputCol='Tokens')
vectorizer_3 = CountVectorizer(inputCol='Tokens',outputCol='Features')

# Split data set into train and test sets
train_data, test_data = df_data.randomSplit((0.8,0.2), seed=42)

# ML model 
rfc_3 = RandomForestClassifier(featuresCol='Features',labelCol='Label')

# Pipeline
pipeline_3 = Pipeline(stages=[tokenizer_3,vectorizer_3,rfc_3])

# Tuning Parameters
paramGrid_3 = (ParamGridBuilder()
        .addGrid(rfc_3.maxDepth, [2, 5, 10]) # Default max depth = MaxDepth=5
        .addGrid(rfc_3.maxBins, [5, 20, 32]) # Default max bins = 32
        .addGrid(rfc_3.numTrees, [5, 20, 50]) # Default numTrees = 20
        .build())

# Multiclass Classification Evaluator    
evaluator_3 = MulticlassClassificationEvaluator(labelCol='Label',
                                              predictionCol='prediction',
                                              metricName='accuracy')

# Cross Validator - creating with 2 folds
crossVal_3 = CrossValidator(estimator = pipeline_3, 
                          estimatorParamMaps = paramGrid_3, 
                          evaluator = evaluator_3, 
                          numFolds = 2)
# Implementation
model_3 = crossVal_3.fit(train_data)

# Predictions
predictions_3 = model_3.transform(test_data)

# Accuracy Score
accuracy_model_3 = evaluator_3.evaluate(predictions_3)

print(accuracy_model_3)

0.5864197530864198


In [0]:
# pulling the hyper parameter selected
bestModel_3 = model_3.bestModel
max_depth_value_3 = bestModel_3.stages[2].getMaxDepth()
max_bins_value_3 = bestModel_3.stages[2].getMaxBins()
num_trees_value_3 = bestModel_3.stages[2].getOrDefault("numTrees")
hyper_parameter_3 = [max_depth_value_3, max_bins_value_3, num_trees_value_3]
result_3 = ("maxDepth: "+ str(hyper_parameter_3[0])+", maxBins: "+ str(hyper_parameter_3[1])+", numTrees: "+ str(hyper_parameter_3[2]))
print(result_3)

maxDepth: 10, maxBins: 5, numTrees: 50


**Pipeline Implementation 4: TF-IDF + Logistic Regression**

In [0]:
Pipeline_4 = "TF-IDF + Logistic Regression"
# Feature extraction (TF-IDF)
tokenizer_4 = Tokenizer(inputCol='Preprocessed',outputCol='Tokens')
hashing_4 = HashingTF(inputCol='Tokens',outputCol='rawFeatures')
idf_4 = IDF(inputCol='rawFeatures',outputCol='Features')

# Split data set into train and test sets
train_data, test_data = df_data.randomSplit((0.8,0.2), seed=42)

# ML model
lr_4 = LogisticRegression(featuresCol='Features',labelCol='Label')

# Pipeline
pipeline_4 = Pipeline(stages=[tokenizer_4,hashing_4,idf_4,lr_4])

# Tuning Parameters
paramGrid_4 = (ParamGridBuilder()
        .addGrid(lr_4.regParam, [0.0, 0.3, 0.5]) # regularization parameter (default is 0)
        .addGrid(lr_4.elasticNetParam, [0.0, 0.1, 0.2]) # Elastic Net Parameter (default is 0)
        .build())

# Multiclass Classification Evaluator    
evaluator_4 = MulticlassClassificationEvaluator(labelCol='Label',
                                              predictionCol='prediction',
                                              metricName='accuracy')

# Cross Validator - creating with 2 folds
crossVal_4 = CrossValidator(estimator = pipeline_4, 
                          estimatorParamMaps = paramGrid_4, 
                          evaluator = evaluator_4, 
                          numFolds = 2)
# Implementation
model_4 = crossVal_4.fit(train_data)

# Predictions
predictions_4 = model_4.transform(test_data)

# Accuracy Score
accuracy_model_4 = evaluator_4.evaluate(predictions_4)

print(accuracy_model_4)

0.5679012345679012


In [0]:
# pulling the hyper parameter selected
bestModel_4 = model_4.bestModel
reg_param_value_4 = bestModel_4.stages[3].getRegParam()
elastic_net_value_4 = bestModel_4.stages[3].getElasticNetParam()
hyper_parameter_4 = [reg_param_value_4, elastic_net_value_4]
result_4 = ("regParam: "+ str(hyper_parameter_4[0])+", elasticNetParam: "+ str(hyper_parameter_4[1]))
print(result_4)

regParam: 0.3, elasticNetParam: 0.1


**Pipeline Implementation 5: TF-IDF + Naive Bayes**

In [0]:
Pipeline_5 = "TF-IDF + Naive Bayes"
# Feature extraction (TF-IDF)
tokenizer_5 = Tokenizer(inputCol='Preprocessed',outputCol='Tokens')
hashing_5 = HashingTF(inputCol='Tokens',outputCol='rawFeatures')
idf_5 = IDF(inputCol='rawFeatures',outputCol='Features')

# Split data set into train and test sets
train_data, test_data = df_data.randomSplit((0.8,0.2), seed=42)

# ML model
nb_5 = NaiveBayes(featuresCol='Features',labelCol='Label')

# Pipeline
pipeline_5 = Pipeline(stages=[tokenizer_5,hashing_5,idf_5,nb_5])

# Tuning Parameters - adjust smoothing
paramGrid_5 = (ParamGridBuilder()
        .addGrid(nb_5.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]) # Default smoothing = 1.0
        .build())

# Multiclass Classification Evaluator    
evaluator_5 = MulticlassClassificationEvaluator(labelCol='Label',
                                              predictionCol='prediction',
                                              metricName='accuracy')

# Cross Validator - creating with 2 folds
crossVal_5 = CrossValidator(estimator = pipeline_5, 
                          estimatorParamMaps = paramGrid_5, 
                          evaluator = evaluator_5, 
                          numFolds = 2)
# Implementation
model_5 = crossVal_5.fit(train_data)

# Predictions
predictions_5 = model_5.transform(test_data)

# Accuracy Score
accuracy_model_5 = evaluator_5.evaluate(predictions_5)

print(accuracy_model_5)

0.5617283950617284


In [0]:
# pulling the hyper parameter selected
bestModel_5 = model_5.bestModel
smoothing_value_5 = bestModel_5.stages[3].getSmoothing()
hyper_parameter_5 = [smoothing_value_5]
result_5 = ("smoothing: "+ str(hyper_parameter_5[0]))
print(result_5)

smoothing: 1.0


**Pipeline Implementation 6: TF-IDF + RandomForest**

In [0]:
Pipeline_6 = "TF-IDF + Random Forest"
# Feature extraction (TF-IDF)
tokenizer_6 = Tokenizer(inputCol='Preprocessed',outputCol='Tokens')
hashing_6 = HashingTF(inputCol='Tokens',outputCol='rawFeatures')
idf_6 = IDF(inputCol='rawFeatures',outputCol='Features')

# Split data set into train and test sets
train_data, test_data = df_data.randomSplit((0.8,0.2), seed=42)

# ML model
rfc_6 = RandomForestClassifier(featuresCol='Features',labelCol='Label')

# Pipeline
pipeline_6 = Pipeline(stages=[tokenizer_6,hashing_6,idf_6,rfc_6])

# Tuning Parameters
paramGrid_6 = (ParamGridBuilder()
        .addGrid(rfc_6.maxDepth, [2, 5, 10]) # Default max depth = MaxDepth=5
        .addGrid(rfc_6.maxBins, [5, 20, 32]) # Default max bins = 32
        .addGrid(rfc_6.numTrees, [5, 20, 50]) # Default numTrees = 20
        .build())

# Multiclass Classification Evaluator    
evaluator_6 = MulticlassClassificationEvaluator(labelCol='Label',
                                              predictionCol='prediction',
                                              metricName='accuracy')

# Cross Validator - creating with 2 folds
crossVal_6 = CrossValidator(estimator = pipeline_6, 
                          estimatorParamMaps = paramGrid_6, 
                          evaluator = evaluator_6, 
                          numFolds = 2)
# Implementation
model_6 = crossVal_6.fit(train_data)

# Predictions
predictions_6 = model_6.transform(test_data)

# Accuracy Score
accuracy_model_6 = evaluator_6.evaluate(predictions_6)

print(accuracy_model_6)

0.41358024691358025


In [0]:
# pulling the hyper parameter selected
bestModel_6 = model_6.bestModel
max_depth_value_6 = bestModel_6.stages[3].getMaxDepth()
max_bins_value_6 = bestModel_6.stages[3].getMaxBins()
num_trees_value_6 = bestModel_6.stages[3].getOrDefault("numTrees")
hyper_parameter_6 = [max_depth_value_6, max_bins_value_6, num_trees_value_6]
result_6 = ("maxDepth: "+ str(hyper_parameter_6[0])+", maxBins: "+ str(hyper_parameter_6[1])+", numTrees: "+ str(hyper_parameter_6[2]))
print(result_6)

maxDepth: 10, maxBins: 5, numTrees: 5


In [0]:
print (Pipeline_1,": ", accuracy_model_1)
print (Pipeline_2,": ", accuracy_model_2)
print (Pipeline_3,": ", accuracy_model_3)
print (Pipeline_4,": ", accuracy_model_4)
print (Pipeline_5,": ", accuracy_model_5)
print (Pipeline_6,": ", accuracy_model_6)

Bag of Words + Logistic Regression :  0.5679012345679012
Bag of Words + Naive Bayes :  0.5864197530864198
Bag of Words + Random Forest :  0.5864197530864198
TF-IDF + Logistic Regression :  0.5679012345679012
TF-IDF + Naive Bayes :  0.5617283950617284
TF-IDF + Random Forest :  0.41358024691358025


In [0]:
data_final = [[Pipeline_1, accuracy_model_1, result_1],
             [Pipeline_2, accuracy_model_2, result_2],
             [Pipeline_3, accuracy_model_3, result_3],
             [Pipeline_4, accuracy_model_4, result_4],
             [Pipeline_5, accuracy_model_5, result_5],
             [Pipeline_6, accuracy_model_6, result_6]]

columns_final = ["Feature Extraction and Classifier", "Accuracy Score", "Hyperparameters"]
df_final = spark.createDataFrame(data_final, columns_final)
display(df_final)

Feature Extraction and Classifier,Accuracy Score,Hyperparameters
Bag of Words + Logistic Regression,0.5679012345679012,"regParam: 0.3, elasticNetParam: 0.1"
Bag of Words + Naive Bayes,0.5864197530864198,smoothing: 1.0
Bag of Words + Random Forest,0.5864197530864198,"maxDepth: 10, maxBins: 5, numTrees: 50"
TF-IDF + Logistic Regression,0.5679012345679012,"regParam: 0.3, elasticNetParam: 0.1"
TF-IDF + Naive Bayes,0.5617283950617284,smoothing: 1.0
TF-IDF + Random Forest,0.41358024691358025,"maxDepth: 10, maxBins: 5, numTrees: 5"
